***GENERATED CODE FOR creditcardfrauddetectionclassification PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
#%run creditcardfrauddetectionclassificationHooks.ipynb
try:
	#sourcePreExecutionHook()

	creditcardtrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/1611443651749_creditcard-train.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line', 'FilePath': '/Finance Models/Credit Card Fraud Detection - Classification/creditcard-train.csv', 'filename': '1611443651749_creditcard-train.csv', 'viewFileName': 'creditcard-train.csv'}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run creditcardfrauddetectionclassificationHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(creditcardtrain,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Time", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "38223.32", "stddev": "1298.86", "min": "36057.0", "max": "40449.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Time"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V1", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.36", "stddev": "1.77", "min": "-12.47723253", "max": "1.562781862", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V2", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.07", "stddev": "1.53", "min": "-11.6900268", "max": "7.383615067", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V3", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.66", "stddev": "1.15", "min": "-6.749516521", "max": "3.17093614", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V4", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.01", "stddev": "1.37", "min": "-3.985170389", "max": "4.378912452", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V4"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V5", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.25", "stddev": "1.23", "min": "-6.300945632", "max": "5.570918167", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V5"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V6", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.13", "stddev": "1.3", "min": "-4.164797021", "max": "4.738257301", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V6"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V7", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.04", "stddev": "1.02", "min": "-10.41569781", "max": "3.88551142", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V7"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V8", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.04", "stddev": "1.12", "min": "-10.44255047", "max": "5.305287617", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V8"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V9", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.18", "stddev": "1.12", "min": "-2.730335934", "max": "6.728079951", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V9"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V10", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.23", "stddev": "1.29", "min": "-11.79718107", "max": "12.17440008", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V10"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V11", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.71", "stddev": "1.21", "min": "-2.032372792", "max": "11.27792073", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V11"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V12", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.17", "stddev": "0.75", "min": "-2.218290393", "max": "3.426308302", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V12"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V13", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.76", "stddev": "1.2", "min": "-2.527993298", "max": "3.312629311", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V13"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V14", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.65", "stddev": "1.39", "min": "-17.72163835", "max": "3.094194675", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V14"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V15", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.13", "stddev": "0.98", "min": "-3.049276681", "max": "2.336733414", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V15"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V16", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.05", "stddev": "0.93", "min": "-10.32201668", "max": "2.863444895", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V16"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V17", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.01", "stddev": "0.72", "min": "-1.784926451", "max": "3.07532563", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V17"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V18", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.03", "stddev": "0.85", "min": "-5.030710038", "max": "4.115559919", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V18"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V19", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.01", "stddev": "0.85", "min": "-2.609280491", "max": "2.458694704", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V19"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V20", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.04", "stddev": "0.55", "min": "-4.82944627", "max": "3.471228631", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V20"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V21", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.03", "stddev": "0.67", "min": "-5.225842448", "max": "6.51990167", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V21"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V22", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.08", "stddev": "0.61", "min": "-3.048648832", "max": "1.788174445", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V22"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V23", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.03", "stddev": "0.3", "min": "-1.552758735", "max": "2.0025512", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V23"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V24", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.0", "stddev": "0.62", "min": "-2.718023955", "max": "1.098310927", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V24"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V25", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.09", "stddev": "0.42", "min": "-1.648416489", "max": "1.609734201", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V25"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V26", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.1", "stddev": "0.54", "min": "-1.024276798", "max": "1.506507416", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V26"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V27", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-0.0", "stddev": "0.36", "min": "-3.077491646", "max": "2.151619524", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V27"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "V28", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.01", "stddev": "0.2", "min": "-1.307731792", "max": "0.971291033", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "V28"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "105.1", "stddev": "209.35", "min": "0.0", "max": "1762.42", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Class", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "Class"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run creditcardfrauddetectionclassificationHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["Time", "V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8", "V9", "V10", "V11", "V12", "V13", "V14", "V15", "V16", "V17", "V18", "V19", "V20", "V21", "V22", "V23", "V24", "V25", "V26", "V27", "V28", "Amount"], "Class")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

